## 什么是图神经网络中的状态更新

对于图神经网络中的基础定义，我们通常用$G=(V, E)$来表示，其中$G$表示图，$V$表示节点，$E$表示边。

`GNN`的学习目标目标：**学习图结构中每个节点的状态信息**。


<img src="../images/GNN.png" width="20%">

定义节点状态更新的过程来描述整张图结构：

$$
h_{v}^{t} = f(x_{v}, x_{c}O[v],  h_{n}^{t-1}e[v], x_{n})
$$

节点$v$在$t$时刻的节点状态$h_{v}^{t}$，是由节点信息 = (节点信息，边信息，邻节点状态，邻节点)所构成的。

以节点$p1$为例，节点$p1$的相邻节点有$p1$和$p3$，$h_{p1}$的节点状态信息由$p1$节点的信息，与$p1$相邻的两条边$p_{(1,2)},p_{(1,3)}$组成。还有邻节点的状态$h_{p2},h_{p3}$，这个节点状态是上一个时刻的节点状态。最后还有邻节点信息。

$$h_{p1} = f(p_{1}, p_{(1,2)}, p_{(1,3)}, h_{p2}, h_{p3}, p2, p3, \cdots)$$

$f$:是状态更新函数，当我们神经网络能够拟合$f$, 我们就能得到图的状态。这个状态更新函数只是GNN最早期最早期的一个模型。

## 开发环境&图描述

<img src="../images/GNN_data.png" width="40%">

上图中是一个有向图网络结构，其中包含了四个顶点。其中A和C是一组顶点，B和D是一组顶点。用y的取值来进行区分。在顶点旁边有顶点的数据，顶点A旁边的数据是[2, 1]。

In [1]:
import torch
from torch_geometric.data import Data

In [2]:
x = torch.tensor([[2, 1], [5, 6], [3, 7], [12, 0]], dtype=torch.float) # data的特征矩阵
y = torch.tensor([0, 1, 0, 1], dtype=torch.float) # 图节点的target
# 边的关系，从0号顶点到1号顶点； 1号顶点到0号顶点；0号顶点到3号顶点，2号顶点到1号顶点; 3号点到2号点。
edge_index = torch.tensor([[0, 1, 0, 2, 3],
                           [1, 0, 3, 1, 2]], dtype=torch.long) # 边的索引数据。

In [4]:
data = Data(x=x, y=y, edge_index=edge_index)
data

Data(edge_index=[2, 5], x=[4, 2], y=[4])

查看节点数量：

In [5]:
data.num_nodes # 查看节点数量

4

In [6]:
data.num_edges # 查看边

5

In [7]:
data.num_node_features # 查看节点特征唯独

2

In [8]:
data.contains_isolated_nodes()  # 是否有独立节点

False

In [9]:
data.contains_self_loops() # 是否有环

False

In [10]:
data.is_directed() # 是否是有向图

True

## Geometric构建数据集

构建数据集有两种：

1. 继承InMemoryDataset，一次性加载所有数据到内存；
2. 继承Dataset，分次加载到内存。

在自定义的Dataset的初始化方法中传入数据存放的路径，然后PyG会在这个路径下再划分两个文件夹：

1. raw_dir: 存放原始数据的路径，(一般是CSV、mat等格式)。
2. processed_dir: 存放处理后的数据，一般是pt格式，由process()方法实现。

但是在Pytorch中，实际上是没有这两个文件夹的。

Mini-Batching: 将一组样本组合成一个统一的表示形式，进行并行处理。